# Case Study 6:  Bhargava Cubes

The purpose of this worksheet is to develop a python *class*.  When you have a complicated math thing that you want to manipulate in various ways, a class is a useful container that can have both attributes (data) and methods (functions) in it.  Each instance of a class is an *object*.  As an example of classes in Sage, we have elliptic curves, number fields, rings.  Let's play with these for a minute.

Each class has a *constructor*, or a way to build an *instance* of the thing.  An instance has its own identity, e.g. for the elliptic curve class, an instance is a particular elliptic curve, not the abstract concept of a curve.  We call an instance an *object*.

Here's some ways to initialize an instance of a class.  You call the name of the class as if it were a function, with inputs whatever data is needed to define the object.  In this case the classes are `EllipticCurve` and `NumberField`.

In [ ]:
# the list is a list of coefficients of the Weierstrass form
E = EllipticCurve([0,1]); E

In [ ]:
# this funny notation sets `a` as the name YOU (the user) can type to refer to the root of the polynomial
F.<a> = NumberField(x^2+1); F

A class has its own *methods*, which are functions, and *attributes*, which are data.  You can access these with a dot on the object, for example:

In [ ]:
E.torsion_points()

In [ ]:
F.galois_group()

Something like the discriminant of a field might be considered an attribute, but Sage prefers only to expose methods, so you'll see the `()` on even basic data.

In [ ]:
F.discriminant()

In [ ]:
E.discriminant()

In this worksheet, we'll build a class that represents a *Bhargava cube*.  A Bhargava cube is a $2 \times 2 \times 2$ cube of integers.  Here's the basic syntax to declare a python class that can be constructed.

In [7]:
class BhargavaCube:
    def __init__(self, x):      # constructor is always called `__init__`, the first argument is always `self`, followed by setup data
        self.entries = x        # attribute `entries` stores data about the instance

Let's test it.  Create an instance.

In [ ]:
B = BhargavaCube("whatever")

What does your instance look like?

In [ ]:
B

The code above isn't great, because it doesn't check that x has any particular type.  In fact, right now it's just a box that stores anything.  In fact, you may have created your instance with a string, a number, an array... (try!)  We want this to be a cube of integers, so maybe should add some type checking and some formatting for the output.  What we'd really like is for our constructor to properly handle either of the two forms of input `[a,b,c,d,e,f,g,h]` (flat list) or `[[[a, b],[c, d]],[[e, f],[g, h]]]` (nested lists) as input representing the cube of this form:
```
      front: [[a, b],
             [c, d]]
      rear: [[e, f],
             [g, h]]
```

The main principle to designing something like this is EAFP:  Easier to Ask Forgiveness than Permission.  So the idea is rather than make super complicated code that will check whether the input satisfies all kinds of properties before trying to put it into the desired box (asking permission), just try to stuff it in and if anything breaks, report the error (hope for forgiveness).  

As for stuffing things in the box, Python has nice more advanced ways to handle assignment.  Try the following example cells to learn some of them.

In [ ]:
a,b = [1,2]
print(a,b)

In [ ]:
[a,b] = [1,2]
print(a,b)

In [ ]:
((a,b),(c,d)) = [[1,2],[3,4]]
print(a,b,c,d)

This means you can anticipate the nested structure of something and slot all your various entries into named variables.

Ok, next up is the concept of `try ... except`.  This is a way to handle errors that your commands cause.  Example, what will this code do?

In [ ]:
for i in range(-3,3):
    try:
        x = 1/i
        print(x)
    except:
        pass

Modify the code above so that instead of ignoring the error (`pass`) it prints the word "infinity".

Maybe instead of returning infinity, you'd rather have the code return an error (also called `exception`), then it looks like this:

In [ ]:
for i in range(-3,3):
    try:
        x = 1/i
        print(x)
    except Exception as e: # You can use this as a magic formula for now; python has lots of inbuilt error handling
        raise Exception("Bah!  Don't divide by zero!") from e

Before we go modifying our class, here's some further useful functionality:

1. `list(x)` turns x into a list if that's a thing you can do to x (maybe it was a tuple, or a string)
2. `len(x)` returns the length of x, but this behaves various ways on various types (try it)

In [ ]:
# mess around with types
len("123")

In [ ]:
len([1,2,3])

In [ ]:
len(123)

In [ ]:
len([[1,2,3]])

In [ ]:
list("[[1,2,3]]")

In [ ]:
list((1,2))

In [ ]:
list(matrix([[1,2],[3,4]]))

Ok, with those abilities, we can now add something to our initialization code that tries to interpret the intput as either a list of 8 integers or nested structure with 8 integers, and returns an error if it fails.  I'm going to give you a little structure for this:

1. make an internal function called `_normalize(self, x)` (underscore means it's for internal use), which will take in `x` and try to stuff it into our box:  turn it into a list of 8 things.
2. in `__init__` we want to call `self._normalize(x)` to do that work, before storing the result
3. the function `_normalize` should be a big `try...except` so that any problems will get caught
4. inside `try`, first make x into a list and then if it's length 8, return that
5. if it's not length 8, assume it's in the other form and assign a,b,c,d,e,f,g,h appropriately (we practiced above) to return
6. I've written the `except` clause for you with some explanation

In [22]:
class BhargavaCube:

    def __init__(self, x): # if you want access to yourself, you pass yourself as the first argument
        flat = self._normalize(x)  # subroutine to shape check + flatten to list of length 8
        self._entries = tuple(flat)  

    def _normalize(self, x): # internal attributes and functions begin with underscore (meant not to be used by user)
        try:
            # put stuff
        except (ValueError, TypeError) as e:
                # wrong lengths or non-iterables at some level
                # by just passing, we end up sending all errors to the last TypeError
                pass
        # if we get here, something went wrong so give a general error
        raise TypeError("Expected 8 entries (flat) or a nested 2x2x2 array.")

In [ ]:
# this should work
B = BhargavaCube([1,2,3,4,5,6,7,8])
C = BhargavaCube([[[1,2],[3,4]],[[5,6],[7,8]]])

In [ ]:
# but this should not
D = BhargavaCube([1,2,3,4],[6,7,8,9])

In [ ]:
# and neither should this
D = BhargavaCube([[1,2,3,4],[6,7,8,9]])

In [ ]:
# this works, which is because strings are treated a lot like lists
E = BhargavaCube("12345678")

In [ ]:
# and even more weirdly, this still works
F = BhargavaCube(["hi",3,[2,3],False,5,6,7,8])

So we probably don't want that last thing to work.  So now we can try to make sure the class coerces the entries to integers.  To check the type of something, we can do this.

In [ ]:
n = 3
n.parent()

To (attempt to) change the type of something, you just wrap it.

In [ ]:
m = QQ(n) # change n into a rational (QQ)
print(m) # I want to display two things in this cell, but only the last command gets its result printed, so here I print
m.parent() # but here I don't

Try coercing your rational number 3 back to an integer 3.

In [ ]:
# will this work?
p = ZZ(3/2)

So the code we had already for our class was getting 8 things, but not making them integers.  Make them integers!

While you're at it, add some documentation to your code.  You can use # for code comments, and you can put code descriptions in between `"""` (these are called docstrings).  When you add docstrings to classes and functions, python offers ways to get this info, like `help(object)`.

In [32]:
class BhargavaCube:
    # The following is a description of your class and how it is used.
    """
    PUT STUFF HERE.
    """
    
    def __init__(self, x):
        flat = self._normalize(x)  # subroutine to shape check + flatten to list of length 8
        # put a coercion attempt here (wrap it in try)

        self._entries = tuple(flat)  

    def _normalize(self, x): 
        """Describe me."""
        try:
            lisx = list(x)
            if len(lisx) == 8:
                return lisx
            else:
                ((a,b),(c,d)), ((e,f),(g,h)) = x
                return [a,b,c,d,e,f,g,h]
        except (ValueError, TypeError) as e:
                # wrong lengths or non-iterables at some level
                # by just passing, we end up sending all errors to the last TypeError
                pass
        # if we get here, something went wrong so give a general error
        raise TypeError("Expected 8 entries (flat) or a nested 2x2x2 array.")

In [33]:
# try again
F = BhargavaCube([1,2,3,4,5,6,7,8])

In [ ]:
# try again
F = BhargavaCube(["hi",3,[2,3],False,5,6,7,8])

In [ ]:
# try this
help(BhargavaCube)

Ok, but so far our class doesn't do much.  We need a way to display what the cube looks like at a given moment.  This is defined using the built-in function `__repr__` (like `__init__`, this is a pre-defined function name for a specific task).  We want to add this function to your class.  Some experience with how to display variables will help, so here's a quick experiment cell.

In [ ]:
# check this out!
a, b = 1, 2
mystring = f"I like when my {a} tastes yummy like {b}!" # notice the f
mystring

In [41]:
class BhargavaCube:
    """
    Entries are 8 scalars arranged as two 2x2 layers:
      front: [[a, b],
             [c, d]]
      rear: [[e, f],
             [g, h]]
    Accepts either a flat length-8 array [a,b,c,d,e,f,g,h]
    or a pair of pairs of pairs (nested array) 
         [ [[a, b], [c, d]], [[e, f], [g, h]] ]
    """

    def __init__(self, x):
        flat = self._normalize(x)  # subroutine to shape check + flatten to list of length 8
        try:
            flat = [ZZ(v) for v in flat]
        except Exception as e:
            raise TypeError("Coercion to 8 integers failed.") from e
        self._entries = tuple(flat)  

    def __repr__(self):
        # put something here


    def _normalize(self, x): 
        """Return a flat list of 8 entries; raise error otherwise."""
        try:
            lisx = list(x)
            if len(lisx) == 8:
                return lisx
            else:
                ((a,b),(c,d)), ((e,f),(g,h)) = x
                return [a,b,c,d,e,f,g,h]
        except (ValueError, TypeError) as e:
                # wrong lengths or non-iterables at some level
                # by just passing, we end up sending all errors to the last TypeError
                pass
        # if we get here, something went wrong so give a general error
        raise TypeError("Expected 8 entries (flat) or a nested 2x2x2 array.")

In [ ]:
E = BhargavaCube([1,2,3,4,5,6,7,8])
E

Next up!  This is a cube, so it has three viewpoints.  Think of a Rubik's cube you are holding in your hand.  You can view it from any of six directions, and from any of these directions, there are four ways to orient it.  That's too many.  We actually care just about these three:
```
[a,b]    [e,f]
[c,d]    [g,h]

and

[a,e]    [c,g]
[b,f]    [d,h]

and

[a,c]    [b,d]
[e,g]    [f,h]
```
Essentially, these correspond to cyclically reordering the indices:  if a_{i,j,k} is the entries of the first cube, then a_{j,k,i} is the entries of the rotated cube.  In all cases, the diagonal is $a, h$.  These are the 3d notions of `transpose'.

Now we will implement a method `.rotate(i)` in our class, which will rotate `i` times.  It only depends on i mod 3.

In [44]:
class BhargavaCube:
    """
    Entries are 8 scalars arranged as two 2x2 layers:
      front: [[a, b],
             [c, d]]
      rear: [[e, f],
             [g, h]]
    Accepts either a flat length-8 array [a,b,c,d,e,f,g,h]
    or a pair of pairs of pairs (nested array) 
         [ [[a, b], [c, d]], [[e, f], [g, h]] ]
    """

    def __init__(self, x):
        flat = self._normalize(x)  # subroutine to shape check + flatten to list of length 8
        try:
            flat = [ZZ(v) for v in flat]
        except Exception as e:
            raise TypeError("Coercion to 8 integers failed.") from e
        self._entries = tuple(flat)  

    def __repr__(self):
        a,b,c,d,e,f,g,h = self._entries
        return f"BhargavaCube([{a}, {b}, {c}, {d}, {e}, {f}, {g}, {h}])" 

    def rotate(self, i=1): # by putting a default value for i we allow it to be left off
        # put something here


    def _normalize(self, x): 
        """Return a flat list of 8 entries; raise error otherwise."""
        try:
            lisx = list(x)
            if len(lisx) == 8:
                return lisx
            else:
                ((a,b),(c,d)), ((e,f),(g,h)) = x
                return [a,b,c,d,e,f,g,h]
        except (ValueError, TypeError) as e:
                # wrong lengths or non-iterables at some level
                # by just passing, we end up sending all errors to the last TypeError
                pass
        # if we get here, something went wrong so give a general error
        raise TypeError("Expected 8 entries (flat) or a nested 2x2x2 array.")

In [ ]:
E = BhargavaCube([1,2,3,4,5,6,7,8])
print(E)
E.rotate()
print(E)

Great!  Now, let's compute some things from our BhargavaCube.  There's actually a reason these things are interesting!

If the front and rear slices of the cube are matrices A and B, then we can define a quadratic form:
$$ Q(x,y) = \operatorname{det}(Ax + By) $$
We can do the same in each of the other directions.  So a BhargavaCube gives us a collection of three integral binary quadratic forms.  Here's how we can define a quadratic form in Sage:

In [47]:
R.<x, y> = ZZ[] # this is notation for defining a polynomial ring over ZZ with variables x and y.
B = BinaryQF(x^2 + 2*x*y + 3*y^2) # now x and y mean something so we can make a QF

By the way, you can do things with BQFs, check out some of the functions on them.  How do you get the discriminant?

Add code to your class to compute these three quadratic forms and their discriminants.

Before you jump in, consider that it will be more efficient to compute these once upon initialization and just have functions to access the info (stored as an attribute), instead of computing them every single time someone calls for them.

In [50]:
class BhargavaCube:
    """
    Entries are 8 scalars arranged as two 2x2 layers:
      front: [[a, b],
             [c, d]]
      rear: [[e, f],
             [g, h]]
    Accepts either a flat length-8 array [a,b,c,d,e,f,g,h]
    or a pair of pairs of pairs (nested array) 
         [ [[a, b], [c, d]], [[e, f], [g, h]] ]
    """

    def __init__(self, x):
        
        flat = self._normalize(x)  # subroutine to shape check + flatten to list of length 8
        try:
            flat = [ZZ(v) for v in flat]
        except Exception as e:
            raise TypeError("Coercion to 8 integers failed.") from e
        self._entries = tuple(flat)  
        
        # add code here to compute the three forms and discriminants and store them in attributes
    
      
    def _form(self,a,b,c,d,e,f,g,h):
        R.<x, y> = ZZ[]
        A = matrix([[a,b],[c,d]])
        B = matrix([[e,f],[g,h]])
        form = BinaryQF((A*x + B*y).determinant())
        disc = form.discriminant()
        return form, disc

    def __repr__(self):
        a,b,c,d,e,f,g,h = self._entries
        return f"BhargavaCube([{a}, {b}, {c}, {d}, {e}, {f}, {g}, {h}])"

    def form(self, dir):
        # dir = 0,1,2 representing the direction (there are three ways to slice a cube)
        # add code here to expose the correct quadratic form
        

    def disc(self, dir):
        # returns the discriminant of self.form(dir)
        # add code here to expose the correct discriminant
        

    def rotate(self, i=1): # by putting a default value for i we allow it to be left off
        """Rotate the cube"""
        # remember to update the forms when rotating!!

        # rotate entries
        a,b,c,d,e,f,g,h = self._entries
        self._entries = a,e,b,f,c,g,d,h

        # rotate forms


    def _normalize(self, x): 
        """Return a flat list of 8 entries; raise error otherwise."""
        try:
            lisx = list(x)
            if len(lisx) == 8:
                return lisx
            else:
                ((a,b),(c,d)), ((e,f),(g,h)) = x
                return [a,b,c,d,e,f,g,h]
        except (ValueError, TypeError) as e:
                # wrong lengths or non-iterables at some level
                # by just passing, we end up sending all errors to the last TypeError
                pass
        # if we get here, something went wrong so give a general error
        raise TypeError("Expected 8 entries (flat) or a nested 2x2x2 array.")

In [ ]:
E = BhargavaCube([1,2,3,4,5,60,7,8])
E

In [ ]:
E.form(0)

In [ ]:
E._forms

In [ ]:
E._discs

In [ ]:
# let's try rotating
E.rotate(); E

In [ ]:
E._forms

In [ ]:
E._discs

In [ ]:
# now I'm going to initialize the once-rotated cube from scratch
F = BhargavaCube([1, 3, 5, 7, 2, 4, 60, 8])

In [ ]:
# and make sure the forms are the same as if I initialized and then rotated
F.forms()

In [ ]:
F._discs

Do you notice anything about the discriminants?  Try more examples, what is your conjecture?

In fact, any integer cube has a well-defined discriminant and represents three quadratic forms of that discriminant.  A famous result in number theory puts quadratic forms of a given discriminant in bijection with ideals of the ring of integers of that discriminant.  So a Bhargava cube represents three ideals from a quadratic ring.  What Bhargava showed is that not only are they from the same ring, but if the ideals are $I_1$, $I_2$, $I_3$ then $I_1 \cdot I_2 \cdot I_3 = id$ in the class group!

If you would like to do more, there is an $\operatorname{SL}(2,\mathbb{Z})$ action on the Bhargava cube by multiplying both slices simultaneously by a matrix from the left.  By rotating, there's three different actions.  Bhargava describes a method of `reduction of cubes' along the lines of reduction of quadratic forms.  His annals paper on this is is very readable.